In [ ]:
%%bigquery model_created
CREATE OR REPLACE MODEL `financial-fraud-detection-1.sec_enforcement_actions.sec_embeddings`
REMOTE WITH CONNECTION `financial-fraud-detection-1.us.vertex_AI_connection`
OPTIONS (ENDPOINT = 'text-embedding-005');

Query is running:   0%|          |

In [ ]:
print(model_created)

Empty DataFrame
Columns: []
Index: []


In [ ]:
%%bigquery view_created
CREATE OR REPLACE VIEW financial-fraud-detection-1.sec_enforcement_actions.view_sec_actions_content AS
SELECT
  id,
  title,
  CONCAT(
    'Title: ', IFNULL(title, ''), '\n',
    'Summary: ', IFNULL(summary, ''), '\n',
    'Tags: ', ARRAY_TO_STRING(tags, ', '), '\n',
    'Violated Sections: ', ARRAY_TO_STRING(violatedSections, ', '), '\n',
    'Complaints: ', ARRAY_TO_STRING(complaints, ' | '), '\n',
    'Penalty Amounts: ', ARRAY_TO_STRING(ARRAY(SELECT CONCAT(p.imposedOn, ': ', CAST(p.penaltyAmount AS STRING), ' (', p.penaltyAmountText, ')') FROM UNNEST(penaltyAmounts) p), '; '), '\n',
    'Entities: ', ARRAY_TO_STRING(ARRAY(
      SELECT CONCAT(
        e.name, ' (', e.role,
        IF(IFNULL(e.ticker, '') != '', CONCAT(', ', e.ticker), ''),
        IF(e.cik IS NOT NULL, CONCAT(', CIK: ', CAST(e.cik AS STRING)), ''), ')'
      )
      FROM UNNEST(entities) e
    ), '; '), '\n',
    'Requested Relief: ', ARRAY_TO_STRING(requestedRelief, ', '), '\n',
    'Parallel Actions By: ', ARRAY_TO_STRING(parallelActionsTakenBy, ', '), '\n',
    'Whistleblower Awards: ', ARRAY_TO_STRING(ARRAY(SELECT w.awardText FROM UNNEST(whistleblowerAwards) w), '; '), '\n',
    'Investigated By: ', ARRAY_TO_STRING(investigationConductedBy, ', '), '\n',
    'Litigation Led By: ', ARRAY_TO_STRING(litigationLedBy, ', '), '\n',
    'Other Agencies: ', ARRAY_TO_STRING(ARRAY(SELECT o.name FROM UNNEST(otherAgenciesInvolved) o), ', '), '\n',
    'Resources: ', ARRAY_TO_STRING(ARRAY(SELECT r.label FROM UNNEST(resources) r), '; ')
  ) AS content
FROM
  financial-fraud-detection-1.sec_enforcement_actions.sec_actions;

Query is running:   0%|          |

In [ ]:
print(view_created)

QueryJob<project=financial-fraud-detection-1, location=US, id=0397e215-6d37-41c7-8d01-34217866a7b2>


In [ ]:
%%bigquery word_count_content
SELECT
    (LENGTH(TRIM(content)) - LENGTH(REPLACE(TRIM(content), ' ', ''))) + 1 AS word_count
FROM
    `financial-fraud-detection-1.sec_enforcement_actions.view_sec_actions_content`;

Query is running:   0%|          |

Downloading:   0%|          |

In [39]:
import pandas as pd
from datasets import load_dataset
import numpy as np

In [ ]:
word_count_content

,word_count
0,53
1,55
2,53
3,54
4,58
...,...
2755,197
2756,332
2757,168
2758,204


In [ ]:
word_count_content.sort_values(by='word_count')

,word_count
91,48
139,49
19,50
27,52
16,52
...,...
2114,512
1389,520
2083,522
2180,653


In [ ]:
# maximum 690 words only

In [26]:
%%bigquery table_creation
CREATE OR REPLACE TABLE `financial-fraud-detection-1.sec_enforcement_actions.sec_enforcement_embeddings` AS (
SELECT *
FROM
  ML.GENERATE_EMBEDDING(
  MODEL `financial-fraud-detection-1.sec_enforcement_actions.sec_embeddings`,
  TABLE `financial-fraud-detection-1.sec_enforcement_actions.view_sec_actions_content`,
  STRUCT(TRUE AS flatten_json_output, 'RETRIEVAL_DOCUMENT' as task_type)
  )
);

Query is running:   0%|          |

In [ ]:
print(table_creation)

In [27]:
%%bigquery embedding_statistics
SELECT ml_generate_embedding_statistics.token_count,
  ml_generate_embedding_statistics.truncated,
  ml_generate_embedding_status
FROM `financial-fraud-detection-1.sec_enforcement_actions.sec_enforcement_embeddings`

Query is running:   0%|          |

Downloading:   0%|          |

In [28]:
embedding_statistics

,token_count,truncated,ml_generate_embedding_status
0,116,false,
1,116,false,
2,119,false,
3,123,false,
4,123,false,
...,...,...,...
2755,1126,false,
2756,1269,false,
2757,1393,false,
2758,1532,false,


In [29]:
embedding_statistics['truncated'].unique()

array(['false'], dtype=object)

In [30]:
embedding_statistics['ml_generate_embedding_status'].unique()

array([''], dtype=object)

In [33]:
embedding_statistics['token_count'].astype(int).max()

1539

In [34]:
embedding_statistics['token_count'].astype(int).min()

116

In [38]:
%%bigquery vector_creation
CREATE OR REPLACE VECTOR INDEX `sec_index`
ON `financial-fraud-detection-1.sec_enforcement_actions.sec_enforcement_embeddings` (embedding_v1)
OPTIONS(distance_type='COSINE', index_type='IVF')

Executing query with job ID: 7ca7880d-39f9-4793-bd55-ee7aea0783ce
Query executing: 0.41s


ERROR:
 400 GET https://bigquery.googleapis.com/bigquery/v2/projects/financial-fraud-detection-1/queries/7ca7880d-39f9-4793-bd55-ee7aea0783ce?maxResults=0&location=US&prettyPrint=false: Unrecognized name: embedding_v1 at [2:86]

Location: US
Job ID: 7ca7880d-39f9-4793-bd55-ee7aea0783ce



In [ ]:
test_2010 = load_dataset(
    "c3po-ai/edgar-corpus",
    name="year_2010",     # config name for year-2010 subset
    split="test"
)

In [43]:
dataset = load_dataset('c3po-ai/edgar-corpus')

Dataset.from_dict()

RuntimeError: Dataset scripts are no longer supported, but found edgar-corpus.py